# Code Python - Electre Tri 

## Introduction to the project

The company at the origin of the request is the social landlord 3F, part of the national group Action Logement. Its request concerns the renovation of three of its housing buildings located in the Lyon region. These buildings having been built in the years 2014, the company thus considered necessary to carry out renovation works for the whole of these 3 buildings. 

The company has found it difficult in the past to find out how to renovate a building in view regarding the different aspects that come into play. For example, when looking at the minimum energy loss between primary and final energy, gas appears to be the most interesting form of energy. However, when looking at the environmental impact of this form of energy, gas is badly ranked. Thus, they wish to take into account several aspects of energy renovation in this project.  

Since several options of renovation are possible and the decision is based on multiple criteria, it has been chosen thaht a multi-criteria analysis should be carried out.


### Electre Tri as a multi-criteria analysis 

The Electre Tri method is the multi-criteria analysis that is selected for the project. In its process, the input data for each item is normalised using thresholds and compared to difference profiles that separate categories. This method results in an optimistic and pessimistic ranking of the elements in which every actions are ranked in categories.

The following code allows to execute step by step the calculations of the Electre Tri method :

*expliquer peut être comment fonctionne la méthode vite fait en mode il y a 28 scenarios 16 critères et il faut poids thresholds etc*

In [322]:
import csv
import pandas as pd
import numpy as np
from numpy import random, vstack, empty
import math


### Import of data from csv file as a Pandas Dataframe

The input of the whole analysis is a csv file containing the following informations : 
- The mean value of the performance of each scenario regarding each criteria 
- The weight of each criteria 
- The variance of each criteria
- The 5 reference profiles : b0, b1, b2, b3, b4 and b5 
- The 3 thresholds : q (the indiference threshold), p (the preference threshold), v (the veto threshold)

It is imported as a dataframe `d`.


In [323]:
d = pd.read_csv('Input_data.csv')
λ = 0.75

### Monte Carlo Function

Since we study 28 scenarios according to 16 criteria, compute all the possible combinations of performance values would not be feasible. To have more robust results, our hypothesis is to use the Monte Carlo method to obtain data sets from distributions and use those data sets in the Electre Tri procedure. 

Monte-Carlo simulation is used in complex systems in order to estimate some operations by using random sample and statistical modeling. 
1. Pick a value from Probability Distribution Functions
2. Run the calculation multiple times: Electre Tri in our case
3. Obtain a set of results to be analyzed 

The first step involve to be given Probability Distribution Functions as inputs. For our study, all the values will be represented as normal distributions. To descrive these distributions 2 parameters are needed : 
- the mean value 
- the variance

*insert an image of a normal distribution showing the mean value & the variance*

These values are given in the `d` DataFrame given as input of the code. 

The following function allows to :
1. Thanks to the variance and mean value for each performance : create the Normal Distribution
2. Pick a random value in each of it
3. Return a DataFrame also called `d` with the random values picked 




In [324]:
def MCarlo(d):
    for i in d.index:
        variance = d['VAR'][i]
        for j in d.iloc[:, 0:28]:
            m = d[j][i]
            v = abs(m*variance)
            perf = random.normal(m, v, 1)
            d[j][i] = perf[0]
    return d




autre version optimisée :

In [325]:
def MCarlo(d):
    variance = d['VAR'].values
    m = d.iloc[:, 0:28].values
    v = np.abs(m * variance[:, np.newaxis])
    perf = np.random.normal(m, v)
    d.iloc[:, 0:28] = perf
    return d


### Concordance

The concordance matrix is a table that compare each pair of alternatives being considered, in our case, the sceanarios. In other words, it evaluates how well each option performs relative to the others with respect to the set of criteria. 

This function take as input the DataFrame containig all the performances as well as all the others parameters and input of the method, but only the performances, the reference profiles, and the thresholds will be used.

The objective is to calculate the concordance between each pair of alternative and reference profiles and in both ways: 
- The concordance $C_j(a_i,b_k)$
- The concordance $C_j(a_i,b_k)$ <br>
*for $i$ the scenarios, $k$ the reference profiles and $j$ the criteria*

Here is how the two types of concordance are calculated in the function: <br>
<center>

$C_j(a_i,b_k) = uj(a_i)-u_j(b_j)+p_j/p_j-q_j$<br>
$C_j(b_k,a_i) = uj(b_j)-u_j(a_i)+p_j/p_j-q_j$<br>

</center>


If the value is higher than one it is replaced by one, and if it is smaller dans zero it is replaced by zero. 

Finally, the function returns two DataFrames : 
- `dconca` : The concordance between the performances and the reference profiles $C_k(a_i,b_j)$
- `dconcb` : The concordance between the reference profiles and the performances $C_k(a_i,b_j)$


In [326]:
def conce2(d):
    def calculate_alpha_beta(col1, col2, w, x, y):
        alpha = (col1 - col2 + w) / (w - x)
        beta = (col2 - col1 + w) / (w - x)
        return pd.DataFrame({"alpha": alpha.clip(0, 1), "beta": beta.clip(0, 1)})

    w = d[d.columns[37]]
    x = d[d.columns[36]]
    y = d.iloc[:,30:36]
    alpha_beta = d.iloc[:, 0:28].apply(calculate_alpha_beta, args=(y, w, x, y))
    new_df = alpha_beta["alpha"]
    new_df2 = alpha_beta["beta"]

    return new_df, new_df2


### Discordance

The discordance matrix is a matrix that is used to represent the degree of discordance between pairs of alternatives. It is typically constructed by comparing the values of each alternative on each criterion, and determining whether the difference between the values is significant enough to cause discordance. 

The objective is to calculate the discordance between each pair of alternative and reference profiles and in both ways: 
- The discordance $D_j(a_i,b_k)$
- The discordance $D_j(b_k,a_i)$ <br>
*for $i$ the scenarios, $k$ the reference profiles and $j$ the criteria*

Here is how the two types of discordance are calculated in the function: <br>
<center>

$D_j(a_i,b_k) = uj(b_k)-u_j(a_i)-p_j/v_j-p_j$<br>
$D_j(b_k,a_i) = uj(a_i)-u_j(b_k)-p_j/v_j-p_j$<br>

</center>


If the value is higher than one it is replaced by one, and if it is smaller dans zero it is replaced by zero. 


Finally, the function returns two DataFrames : 
- `ddiscoa` : The discordance between the performances and the reference profiles $D_j(a_i,b_k)$
- `ddiscob` : The discordance between the reference profiles and the performances $D_j(b_k,ba_i)$

In [327]:


def disco2(d):
    new_df = pd.DataFrame()
    new_df2 = pd.DataFrame()
    for column in d.iloc[:, 0:28]:
        for col in d.iloc[:,30:36]:
            alpha = (d[col]-d[column]-d[d.columns[37]])/(d[d.columns[38]]-d[d.columns[37]])
            beta = (d[column]-d[col]-d[d.columns[37]])/(d[d.columns[38]]-d[d.columns[37]])
            new_df[column+col+' a,b'] = alpha
            new_df2[column+col+' b,a'] = beta
    new_df[new_df<0]=0
    new_df[new_df>1]=1
    new_df2[new_df<0]=0
    new_df2[new_df>1]=1
    return new_df, new_df2



### Global concordance

The function allows to calculate the global concordance of each scenario regarding each threshold. It takes as input the concordance matrix and the weights for each criteria. <br>
*explain what si the global concordance for*

The function takes as input the weights of each criterion, located in the `d` DataFrame as well as the concordance matrix, separated into 2 DataFrames previously : `dconca` and `dconcb`. 

The objective is, for each scenario calculate the following global concordance : 

<center>

$C(a_i,b_k) = \frac {\sum_{j} C_j(a_i,b_k) * w_j}{\sum_{j} w_j}$

</center>

*with i the scenarios and k the reference profiles*

This function has to be used twice :
- Once taking as input `dconca` and returning as output `dgconca` : $C(a_i,b_k)$
- Once taking as input `dconcb` and returning as output `dgconcb` : $C(b_k,a_i)$








autre version optimisée je sais pas encore laquelle choisir

In [328]:
def global_conc(d,dconc1):
    new_df = pd.DataFrame(index=['b0', 'b1', 'b2', 'b3', 'b4', 'b5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4']) 
    cols = dconc1.columns
    i = 0
    for j in range(0, len(cols),6):
        a = sum(dconc1[cols[j]]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        b = sum(dconc1[cols[j+1]]*d[d.columns[28]])/sum(d[d.columns[28]])  
        c = sum(dconc1[cols[j+2]]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        dr = sum(dconc1[cols[j+3]]*d[d.columns[28]])/sum(d[d.columns[28]])  
        e = sum(dconc1[cols[j+4]]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        f = sum(dconc1[cols[j+5]]*d[d.columns[28]])/sum(d[d.columns[28]]) 
        th = [a,b,c,dr,e,f]
        new_df[new_df.columns[i]]= th
        i = i+1
    return new_df

v2

### Degree of credibility

The degree of credibility represents *dire ce que c'est concrètement*

The degree of credibility is calculated thanks to :
- the global concordance of each scenario with each reference profile $C(a_i,b_k)$ named `dgconca`
- the discordance matrix, separated in two DataFrames `ddiscoa` and `ddiscob`

The objective is, for each scenario, follow these steps : 

If for all the criteria $j$,   $D_j(a_i,b_k) \le C(a_i,b_k)$:
<center>

$ \delta(a_i,b_k) = C(a_i,b_k) $

</center>

Else : 

<center>

$ \delta(a_i,b_k) = C(a_i,b_k) * \prod_{j \in J } \frac{(1-D_j(a_i,b_k))}{(1-C(a_i,b_k))} $

</center>

*With J : all the criteria for whom  $D_j(a_i,b_k) \ge C(a_i,b_k)$*

The following function should be run 2 times : 
- Once considering the comparaison of performance with reference profiles $(a_i,b_k)$
    - input : `dgconca` : the global performance  $C(a_i,b_k) $ and `ddiscoa` : the discordance :  $D_j(a_i,b_k) $
    - output : `dcreda`: the credibility $ \delta(a_i,b_k)$ <br>
    

- Once considering the comparaison of reference profiles with performances $(b_k,a_i)$
    - input : `dgconcb` : the global performance  $C(b_k,a_i) $ and `ddiscoa` : the discordance :  $D_j(b_k,a_i) $
    - output : `dcredb` the credibility $ \delta(b_k,a_i)$

In [329]:
def credibility(dgconca, ddisc):
    dcred = dgconca.copy()
    cols = ddisc.columns
    for j in range(0, len(cols),6):           #pour toutes les colonnes de discordance,toutes les 6 colonnes donc pour chaque scénario
        ddiscj = [ddisc[ddisc.columns[j]], ddisc[ddisc.columns[j+1]], ddisc[ddisc.columns[j+2]], ddisc[ddisc.columns[j+3]], ddisc[ddisc.columns[j+4]], ddisc[ddisc.columns[j+5]]].copy()
        cglobal = [dgconca[dgconca.columns[j/6]][0], dgconca[dgconca.columns[j/6]][1],dgconca[dgconca.columns[j/6]][2], dgconca[dgconca.columns[j/6]][3], dgconca[dgconca.columns[j/6]][4], dgconca[dgconca.columns[j/6]][5]]
        dc = [0, 0, 0, 0, 0, 0]
        for i in range(len(cglobal)):        #pour chaque profil de référence
            verif = 0
            for c in ddisc.index:           #parcours les valeurs de la colonne
                if  ddisc[cols[j+i]][c] > cglobal[i]:           #si une valeur de la colonne est supérieur au coef de concordance global 
                    verif = verif + 1
            if verif == 0 :
                dc[i] = cglobal[i]
            else: 
                df_mask = ddiscj[i]>cglobal[i] 
                filtered_ddisc = ddisc[df_mask]
                degree = (((1-filtered_ddisc[cols[j+i]])/(1-cglobal[i])).prod())*cglobal[i] #le degré de credibilité du profil i et du scénario j
        dcred[dcred.columns[j/6]] = dc
    return dcred



### Over Ranking

The objective of this step is to establish preference relationships between performance and reference profiles. 
These relationships are established than the degree  of credibility determined just before and thanks to the cutting threshold $\lambda$. 
The value of this cutting threshold can vary, and it value will be discussed later. 

There are 4 types of relationships that can be established between each $a_i$ and each $b_k$
- $a_i$  `I`  $b_k$ : $a_i$  is Indifferent to  $b_k$ 
- $a_i$  `>`  $b_k$ : $a_i$  is prefered to  $b_k$ 
- $a_i$  `<`  $b_k$ : $a_i$  is not prefered to  $b_k$ 
- $a_i$  `R`  $b_k$ : $a_i$  uncomparable to $b_k$ 

This is how this these relationship are determined : 

![](my_image.jpeg)

*insérer le tableau*

The function will return a Dataframe `dranking` containing all these relations between performance and reference profiles.


In [330]:
def over_ranking_relations(creda, credb, λ):
    new_df = creda.copy()
    classementa = creda.apply(lambda x: x-λ)
    classementb = credb.apply(lambda x: x-λ)
    classementa[classementa > 0] = 1  # surclasse
    classementa[classementa < 0] = 0  # ne surclasse pas
    classementb[classementb > 0] = 1
    classementb[classementb < 0] = 0
    mask = (classementa == classementb) & (classementa == 1)
    new_df = new_df.mask(mask, "I")
    mask = (classementa == classementb) & (classementa == 0)
    new_df = new_df.mask(mask, "R")
    mask = (classementb != 0) & (classementa == 0)
    new_df = new_df.mask(mask, "<")
    mask = (classementa != 0) & (classementb == 0)
    new_df = new_df.mask(mask, ">")
    return new_df


## Sorting

The objective of the whole method is to obtain a ranking of the multiple alternatives we have for our problem. 
This method gives two types of rankings : *the pessimistic ranking and the optimistic ranking.*

A median ranking can be obtained as an average of these two rankings.

### Pessimistic sorting

The following function permits to obtain the pessimistic ranking thanks to the over ranking relationships we just established.

This is how the ranking works : <br>

The 6 reference profiles $b0, b1, b2, b3, b4$ and $b5$ delineate 5 categories : <br>
$C1, C2, C3, C4$ and $C5$, C5 being the best one and C1 the worse : 

*insert image of pessimistic ranking*

For each scenario, these categories will be browsed from the best to the worst ( from C5 to C1 ). 
For each reference profiles encountered the credibility $ \delta(a_i,b_k)$ will be compared to the cutting threshold $\lambda$ : 
- if $ \delta(a_i,b_k) > \lambda $ : the scenario is ranked in the category with the same number as $b_k$
- if $ \delta(a_i,b_k) < \lambda $ : we continue to the next reference profile 



In [331]:
def pessimistic_sort(dov,new_df):
    # transpose the DataFrame so that the rows become columns
    #stop_rows = []  # store the index of the row where the loop stopped for each column
    cat = new_df.index
    for col in dov: #pour le scéénario col  
        etape = new_df[col] 
        print(col)
        for j in reversed(range(len(dov.index))): 
            if dov[col][j] == '>' or dov[col][j] == 'I':
                etape[etape.index[j]] = etape[etape.index[j]] +1
                #new_df[col][new_df.index[j]] = new_df[col][new_df.index[j]]+1
                break
        new_df[col] = etape
        print(new_df[col])
    return new_df #, stop_rows



### Optimistic sorting

The following function permits to obtain the optimistic ranking thanks to the over ranking relationships we just established.

This is how the ranking works : <br>

As previously 6 reference profiles delineate 5 categories, C5 being the best one and C1 the worse : 

*insert image of pessimistic ranking*

The difference is that for this ranking, for each scenario, these categories will be browsed from the worst to the best ( from C1 to C5 ). 
For each reference profiles encountered the over ranking relation will be analyzed : 
- if $a_i$ `<` $b_k$ : the scenario is ranked in the category with the same number as $b_k$
- if $a_i$ `>` $b_k$, $a_i$ `R` $b_k$ or $a_i$ `I` $b_k$ : we continue to the next reference profile 



In [332]:
def optimistic_sort(dov,new_df):
    # transpose the DataFrame so that the rows become columns
    #stop_rows = []  # store the index of the row where the loop stopped for each colum
    cat = new_df.index
    for col in dov: 
        etape = new_df[col] 
        for j in range(len(dov.index)): 
            if dov[col][j] == '<' or dov[col][j] == 'R':
                etape[etape.index[j-1]] = etape[etape.index[j-1]] +1
                break
        new_df[col] = etape 
        print(new_df[col])
    return new_df #, stop_rows



### Electre Tri method

This final method permits to run all the previous methods in order to compute all the steps of the Electre Tri method. 
It takes as input : 
- `d` : the input Dataframe containing the performances, the weights, the variance, the reference profiles and the thresholds
- `rep` : the number of times we want the method to be executed

It will return a csv file containing the repartition of the scenarios in the categories as percentages. 

In [333]:
def electre_tri (d,rep):
    temp = np.zeros((5,28))
    pessi_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    opti_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    for i in range(rep) :
        print(i)
        d = MCarlo(d)
        dconca, dconcb = conce2(d)
        ddisca, ddiscb = disco2(d)
        dgconca = global_conc(d,dconca)
        dgconcb = global_conc(d,dconcb)
        dcreda = credibility(dgconca, ddisca)
        dcredb = credibility(dgconcb, ddiscb)
        dranking = over_ranking_relations(dcreda, dcredb, λ)
        print(dranking)
        pessi_sort = pessimistic_sort(dranking,pessi_sort)
        opti_sort = optimistic_sort(dranking,opti_sort)
        print(pessi_sort)
        print(opti_sort)
    return opti_sort, pessi_sort

In [334]:
def electre_tri2 (d):
    temp = np.zeros((5,28))
    pessi_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    opti_sort = pd.DataFrame(temp, index=['C1', 'C2', 'C3', 'C4', 'C5'], columns=['S1.1','S1.2','S1.3','S1.4','S2.1','S2.2','S2.3','S2.4','S3.1','S3.2','S3.3','S3.4','S4.1','S4.2','S4.3','S4.4','S5.1','S5.2','S5.3','S5.4','S6.1','S6.2','S6.3','S6.4','S7.1','S7.2','S7.3','S7.4'])
    d = MCarlo(d)
    dconca, dconcb = conce2(d)
    ddisca, ddiscb = disco2(d)
    dgconca = global_conc(d,dconca)
    dgconcb = global_conc(d,dconcb)
    dcreda = credibility(dgconca, ddisca)
    dcredb = credibility(dgconcb, ddiscb)
    dranking = over_ranking_relations(dcreda, dcredb, λ)
    pessi_sort = pessimistic_sort(dranking,pessi_sort)
    print(pessi_sort)
    opti_sort = optimistic_sort(dranking,opti_sort)
    print(opti_sort)
    return opti_sort, pessi_sort

In [335]:
repet = 1
opti_sort, pessi_sort = electre_tri (d, repet)

0


ValueError: If using all scalar values, you must pass an index

Ici je pense on peut garder car prends directement les infos des csv mais prendre avec panda et pas faire "append"